[statinfer.com](https://statinfer.com/)

# Packages Installation

In [ ]:
#!pip install nltk
#!pip install spacy

In [ ]:
import pandas as pd
import nltk
import spacy
nltk.download("all")

# Data Importing

In [ ]:
twitter_data=pd.read_csv("https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/Twitter_Sentiment/Twitter_Sentiment_Data.csv")
twitter_data.sample(10)

# A Single function for pre-processing

In [ ]:
contra_Expan_Dict = {"ain`t": "am not","aren`t": "are not","can`t": "cannot","can`t`ve": "cannot have","`cause": "because",
"could`ve": "could have","couldn`t": "could not","couldn`t`ve": "could not have","didn`t": "did not",
"doesn`t": "does not","don`t": "do not","hadn`t": "had not","hadn`t`ve": "had not have","hasn`t": "has not",
"haven`t": "have not","he`d": "he would","he`d`ve": "he would have","he`ll": "he will","he`ll`ve": "he will have",
"he`s": "he is","how`d": "how did","how`d`y": "how do you","how`ll": "how will",
"how`s": "how does","i`d": "i would","i`d`ve": "i would have","i`ll": "i will","i`ll`ve": "i will have","i`m": "i am",
"i`ve": "i have","isn`t": "is not","it`d": "it would","it`d`ve": "it would have","it`ll": "it will","it`ll`ve": "it will have",
"it`s": "it is","let`s": "let us","ma`am": "madam","mayn`t": "may not","might`ve": "might have","mightn`t": "might not",
"mightn`t`ve": "might not have","must`ve": "must have","mustn`t": "must not","mustn`t`ve": "must not have","needn`t": "need not","needn`t`ve": "need not have",
"o`clock": "of the clock","oughtn`t": "ought not","oughtn`t`ve": "ought not have","shan`t": "shall not",
"sha`n`t": "shall not","shan`t`ve": "shall not have","she`d": "she would",
"she`d`ve": "she would have","she`ll": "she will","she`ll`ve": "she will have",
"she`s": "she is","should`ve": "should have","shouldn`t": "should not","shouldn`t`ve": "should not have","so`ve": "so have","so`s": "so is",
"that`d": "that would","that`d`ve": "that would have","that`s": "that is","there`d": "there would","there`d`ve": "there would have","there`s": "there is",
"they`d": "they would","they`d`ve": "they would have","they`ll": "they will","they`ll`ve": "they will have","they`re": "they are","they`ve": "they have",
"to`ve": "to have","wasn`t": "was not"," u ": " you "," ur ": " your "," n ": " and ","won`t": "would not",
"dis": "this","bak": "back","brng": "bring"}

def expanded_form(x):
  if x in contra_Expan_Dict.keys():
    return(contra_Expan_Dict[x])
  else:
    return(x)

from spacy.lang.en.stop_words import STOP_WORDS as spacy_stopwords
spacy_stopwords.update({"would", "rt","like", "ha", "lol", "need", "do"})

import re
def clean_with_re(x):
  x=str(x)
  x=re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'," ", x) #Remove URLs
  x=re.sub(r'[^\w ]+', "", x) # Remove Punctuation-1
  x=re.sub(r"[,!@&\'?\.$%_]"," ", x) # Remove Punctuation-2
  x=re.sub(r"\d+"," ", x) #Remove digits
  return(x)

spacy_model = spacy.load('en_core_web_sm')

def pre_processing(input_data, text_col):
  input_data["text_col_clean"]=input_data[text_col].apply(lambda x:str(x).lower())
  input_data["text_col_clean"]=input_data["text_col_clean"].apply(lambda x:[expanded_form(t) for t in str(x).split()])
  input_data["text_col_clean"]=input_data["text_col_clean"].apply(lambda x:[t for t in x if t not in spacy_stopwords ])
  input_data["text_col_clean"]=input_data["text_col_clean"].apply(lambda x:clean_with_re(x))
  input_data["text_col_clean"]=input_data["text_col_clean"].apply(lambda x:" ".join([t.lemma_ for t in spacy_model(str(x))if t.lemma_ !="-PRON-" ]))
  input_data["text_col_clean"]=input_data["text_col_clean"].apply(lambda x:[t for t in str(x).split() if t not in spacy_stopwords ])
  input_data["text_col_clean"]=input_data["text_col_clean"].apply(lambda x: " ".join(x) )
  print(input_data[[text_col,"text_col_clean"]])

In [ ]:
pre_processing(input_data=twitter_data, text_col="raw_tweet")

# WordClod

In [ ]:
#!pip install wordcloud
from wordcloud import WordCloud
import matplotlib.pyplot as plt
%matplotlib inline

final_text="".join(twitter_data["text_col_clean"])
len(final_text)

plt.figure(figsize = (15, 15), facecolor = None) 
wc=WordCloud(colormap='Set2').generate(final_text)
plt.imshow(wc)
plt.axis("off")
plt.show()

# Document Term Matrix

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

countvec1 = CountVectorizer(min_df= 5)
dtm_v1 = pd.DataFrame(countvec1.fit_transform(twitter_data['text_col_clean']).toarray(), columns=countvec1.get_feature_names(), index=None)
print(dtm_v1.shape)
dtm_v1


# Sentiment Analysis

## Features, Target, Train and Test Data

In [ ]:
twitter_data.info()

In [ ]:
twitter_data['sentiment_label'].value_counts()

In [ ]:
dtm_v1['sentiment_label']=twitter_data['sentiment_label']

#remove neutrals
dtm_v1=dtm_v1[dtm_v1['sentiment_label'] != "neutral"]
print(dtm_v1['sentiment_label'].value_counts())

In [ ]:
from sklearn.model_selection import train_test_split

X=dtm_v1.drop(['sentiment_label'], axis=1)
y=dtm_v1['sentiment_label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2 , random_state=33)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


## Naive Bayes Model

In [ ]:
from sklearn.naive_bayes import MultinomialNB
senti_model = MultinomialNB()
#Fitting model to our data
senti_model.fit(X_train, y_train)

In [ ]:
print("Train Accuracy", senti_model.score(X_train,y_train))
print("Test Accuracy", senti_model.score(X_test,y_test))

In [ ]:
#Prediction
pred_sentiment=senti_model.predict(X_test)
print(pred_sentiment)

## Predicting for new data points

In [ ]:
t1 ="Awesome experience. Go for it. It is a great place"
t2 ="Very bad day for me today. I would like to forget it as soon as possible"
tweet_list=[t1,t2]

In [ ]:
new_comment= pd.DataFrame({"text":tweet_list})

#Spelling Correction
from textblob import TextBlob
new_comment["text_corrected"]=new_comment["text"].apply(lambda x:"".join(TextBlob(x).correct()))
pre_processing(input_data=new_comment, text_col="text_corrected")

In [ ]:
countvec = CountVectorizer()
dtm_newcomment = pd.DataFrame(countvec.fit_transform(new_comment['text_col_clean']).toarray(), columns=countvec.get_feature_names(), index=None)
#print(dtm_newcomment)

dtm_v2=dtm_v1.drop(["sentiment_label"],axis=1)
dtm_newcomment_final=pd.DataFrame(columns=dtm_v2.columns.values)
dtm_newcomment_final=dtm_newcomment_final.append(dtm_newcomment)
dtm_newcomment_final=dtm_newcomment_final.fillna(0)

In [ ]:
print("****Make sure that New DTM and old DTM have same number of columns***")
print("New DTM Shape", dtm_newcomment_final.shape)
print("Overall DTM Shape",dtm_v2.shape)

New_words=[t for t in dtm_newcomment_final.columns.values if t not in dtm_v2.columns.values]
print("Count of new words =>", len(New_words))
print("New words are => ", New_words)

In [ ]:
result=pd.DataFrame()
result["text"]=new_comment["text"]
result["Sentiment"] = senti_model.predict(dtm_newcomment_final)
print(result)

#Dataset-2 : Amazon and Yelp Reviews

Download Amazon and Yelp review data
https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/Amazon_Yelp_Reviews/Review_Data.csv

## Data Importing

In [ ]:
review_data=pd.read_csv("https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/Amazon_Yelp_Reviews/Review_Data.csv")
review_data.sample(10)

## Pre-processing

In [ ]:
pre_processing(input_data=review_data, text_col="Review")

## Word Cloud

In [ ]:
#!pip install wordcloud
from wordcloud import WordCloud
import matplotlib.pyplot as plt
%matplotlib inline

final_text="".join(review_data["text_col_clean"])
len(final_text)

plt.figure(figsize = (15, 15), facecolor = None) 
wc=WordCloud(colormap='rainbow', background_color="grey").generate(final_text)
plt.imshow(wc)
plt.axis("off")
plt.show()

## Document Term Matrix

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

countvec1 = CountVectorizer(min_df= 2)
dtm_v1 = pd.DataFrame(countvec1.fit_transform(review_data['text_col_clean']).toarray(), columns=countvec1.get_feature_names(), index=None)
print(dtm_v1.shape)
dtm_v1


# Sentiment Analysis

## Features, Target, Train and Test Data

In [ ]:
review_data.info()

In [ ]:
review_data

In [ ]:
review_data['Sentiment'].value_counts()

In [ ]:
dtm_v1['sentiment_label']=review_data['Sentiment']

#remove neutrals
dtm_v1=dtm_v1[dtm_v1['sentiment_label'] != "neutral"]
print(dtm_v1['sentiment_label'].value_counts())

In [ ]:
from sklearn.model_selection import train_test_split

X=dtm_v1.drop(['sentiment_label'], axis=1)
y=dtm_v1['sentiment_label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2 , random_state=33)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


## Naive Bayes Model

In [ ]:
from sklearn.naive_bayes import MultinomialNB
senti_model_Amazon = MultinomialNB()
senti_model_Amazon.fit(X_train, y_train)

In [ ]:
print("Train Accuracy", senti_model_Amazon.score(X_train,y_train))
print("Test Accuracy", senti_model_Amazon.score(X_test,y_test))

In [ ]:
'''
Fine-tune alpha - Additive (Laplace/Lidstone) smoothing parameter (0 for no smoothing).
P(W*|Y=1) = P(W*,Y=1)/P(Y=1)
What if W* is not present at training? 
P(W*|Y=1)=Freq of W* present in class y=1 / Freq of class1
P(W*|Y=1)= 0/Freq of class1
Then we may need to add an alpha to both numerator and denominator 
P(W*|Y=1) = (P(W*,Y=1)+alpha)/(P(Y=1)+(total classes)*alpha)
If alpha is too low or zero ==> Overfitting on train data
If alpha is too high ==> Then ignoring too much of training data - Underfitting
'''
senti_model_Amazon = MultinomialNB(alpha=100)
senti_model_Amazon.fit(X_train, y_train)
print("Train Accuracy", senti_model_Amazon.score(X_train,y_train))
print("Test Accuracy", senti_model_Amazon.score(X_test,y_test))

In [ ]:
#Prediction
pred_sentiment=senti_model_Amazon.predict(X_test)
print(pred_sentiment)

## Predicting for new data points

In [ ]:
t1 ="Disappointed with the quality of the product. They need to add more value into it "
t2 ="Never order a product from here. You will regret it. "
t3 = "This restaurent is super good. I am a big fan of their menu " 
tweet_list=[t1,t2,t3]

new_comment= pd.DataFrame({"text":tweet_list})
#Spelling Correction
from textblob import TextBlob
new_comment["text_corrected"]=new_comment["text"].apply(lambda x:"".join(TextBlob(x).correct()))
pre_processing(input_data=new_comment, text_col="text_corrected")

In [ ]:
countvec = CountVectorizer()
dtm_newcomment = pd.DataFrame(countvec.fit_transform(new_comment['text_col_clean']).toarray(), columns=countvec.get_feature_names(), index=None)
#print(dtm_newcomment)
#Now we need to fetch all the columns from the original DTM and fill them with zeros. Fill "really" and "amazed" with 1. 

dtm_v2=dtm_v1.drop(["sentiment_label"],axis=1)
dtm_newcomment_final=pd.DataFrame(columns=dtm_v2.columns.values)
dtm_newcomment_final=dtm_newcomment_final.append(dtm_newcomment)
dtm_newcomment_final=dtm_newcomment_final.fillna(0)

In [ ]:
print("****Make sure that New DTM and old DTM have same number of columns***")
print("New DTM Shape", dtm_newcomment_final.shape)
print("Overall DTM Shape",dtm_v2.shape)

New_words=[t for t in dtm_newcomment_final.columns.values if t not in dtm_v2.columns.values]
print("Count of new words =>", len(New_words))
print("New words are => ", New_words)

In [ ]:
result=pd.DataFrame()
result["text"]=new_comment["text"]
result["Sentiment"] = senti_model_Amazon.predict(dtm_newcomment_final)
print(result)
print("In the training data - \n 1: Positive \n 0: Negative \n")